## Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Import Data

In [2]:
train = pd.read_csv('../cleaneddata/cleanedTraining.csv', index_col=0)

In [4]:
train.describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,LogSales,DayOfWeek,Store,WeekNumber
count,844338.000000,844338.000000,844338.000000,844338.000000,844338.000000,844338.000000,844338.000000,844338.000000,844338.000000,844338.000000,844338.000000,844338.000000,844338.000000,844338.000000,844338.000000,844338.000000,844338.000000,844338.000000,844338.000000
mean,0.553644,0.446356,0.998922,0.000822,0.000172,0.000084,0.806422,0.193578,0.541302,0.018429,0.133795,0.306474,0.526892,0.009722,0.463386,8.757564,3.520350,558.421374,23.646946
std,0.497114,0.497114,0.032812,0.028658,0.013104,0.009170,0.395102,0.395102,0.498292,0.134496,0.340432,0.461029,0.499277,0.098122,0.498658,0.425278,1.723712,321.730861,14.389931
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.828641,1.000000,1.000000,1.000000
25%,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.488588,2.000000,280.000000,11.000000
50%,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,8.759198,3.000000,558.000000,23.000000
75%,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000,9.031214,5.000000,837.000000,35.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,10.634677,7.000000,1115.000000,52.000000


## Train/Test Split

In [6]:
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(train.drop('LogSales', axis=1), 
                                                    train['LogSales'], 
                                                    test_size=0.25, 
                                                    random_state=123)

## Fit a Random Forest

In [7]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor().fit(X_train, y_train)

In [8]:
predictions = rf.predict(X_test)

In [9]:
from sklearn.metrics import mean_squared_error

mean_squared_error(predictions, y_test)

0.029672911893843568

## Got Another Shit Result, Trying AdaBoost

In [13]:
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor

AB = AdaBoostRegressor(RandomForestRegressor(), n_estimators = 100).fit(X_train, y_train)
ab_pred = AB.predict(X_test)

mean_squared_error(ab_pred, y_test)

0.02897686095216984

## Bring in the Evaluation Set

In [123]:
test = pd.read_csv('cleanedtest.csv', index_col=0)
test.head()
# test.describe()

,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,DayOfWeek,Store,WeekNumber,Open,Id
0,0,1,1,0,1,0,0,0,1,0,...,1,0,1,0,0,4,1,38,1,1
1,0,1,1,0,1,0,0,0,1,0,...,1,0,1,0,0,3,1,38,1,857
2,0,1,1,0,1,0,0,0,1,0,...,1,0,1,0,0,2,1,38,1,1713
3,0,1,1,0,1,0,0,0,1,0,...,1,0,1,0,0,1,1,38,1,2569
4,1,0,1,0,1,0,0,0,1,0,...,1,0,1,0,0,7,1,37,0,3425


In [124]:
pred_log_sales = AB.predict(test.drop(['Open','Id'], axis=1))

In [125]:
test['Sales'] = np.exp(pred_log_sales)

In [126]:
test.ix[test['Open'] == 0, 'Sales'] = 0

In [129]:
test.ix[:, ['Id', 'Sales']].to_csv('boostedOneHotRF.csv', index=False, header=['Id', 'Sales'])

In [128]:
test.head()

,0,1,2,3,4,5,6,7,8,9,...,13,14,15,16,DayOfWeek,Store,WeekNumber,Open,Id,Sales
0,0,1,1,0,1,0,0,0,1,0,...,0,1,0,0,4,1,38,1,1,4521.443656
1,0,1,1,0,1,0,0,0,1,0,...,0,1,0,0,3,1,38,1,857,4482.555429
2,0,1,1,0,1,0,0,0,1,0,...,0,1,0,0,2,1,38,1,1713,4381.000000
3,0,1,1,0,1,0,0,0,1,0,...,0,1,0,0,1,1,38,1,2569,4816.971416
4,1,0,1,0,1,0,0,0,1,0,...,0,1,0,0,7,1,37,0,3425,0.000000
